In [33]:
from dataclasses import dataclass
import ipywidgets as widgets
import functools
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import os
import shapemap_tools.utils as utils

import sys

%load_ext autoreload
%autoreload 2

data_path = os.environ.get("DATA_PATH", ".")
conditions_prefix = os.environ.get("CONDITIONS_PREFIX", "")

/tmp/ipykernel_130278/2240647349.py:4: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
class Viewer:
    def update_profile(self, condition, seqid, widget):
        self.toggle_visibility(self.profile_visible, widget, None)
            
        filepath = utils.plot_profiles_svg_pattern.format(path=self.project_path, condition=condition, seqid=seqid)

        try:
            with open(filepath, "rb") as file:
                image = file.read()
                widget.value = image
        except Exception as e:
            widget.value = b''
            widget.layout.display= 'none'
            

            
    def update_aggregated(self, condition, seqid, widget):
        self.toggle_visibility(self.profile_visible, widget, None)
        filepath = utils.plot_pattern.format(path=self.project_path, condition=condition, seqid=seqid)

        try:
            with open(filepath, "rb") as file:
                image = file.read()
                widget.value = image
        except Exception as e:
            widget.value = b''
            widget.layout.display= 'none'

    
    def update_aggregated_full(self, condition, seqid, widget):
        self.toggle_visibility(self.profile_visible, widget, None)
        filepath = utils.plot_full_pattern.format(path=self.project_path, condition=condition, seqid=seqid)
        try:
            with open(filepath, "rb") as file:
                image = file.read()
                widget.value = image
        except Exception as e:
            widget.value = b''
            widget.layout.display= 'none'

            
    def update_structure(self, condition, aptaid, widget):
        self.toggle_visibility(self.structure_visible, widget, None)
        filepath = utils.svg_structure_pattern.format(path=self.project_path, condition=condition, seqid=seqid)
        try:
            with open(filepath, "rb") as file:
                image = file.read()
                widget.value = image
        except:
            widget.value = b''
            widget.layout.display= 'none'

    
    def update_structure_box(self, condition, seqid, widget):
        self.toggle_visibility(self.structure_visible, widget, None)
        children = []
        try:
            if True:
            #for i in range(1, 2):
                filepath = utils.svg_structure_pattern.format(path=self.project_path, condition=condition, seqid=seqid, index="")
                with open(filepath, "r") as file:
                    image = file.read()
                    iw = widgets.HTML(value="<style>div.output_area svg {max-width: 100%; height: 100% !important;}</style>"+ f"<div style='height:800px;width:800px;'>{image}</div>")
                    children.append(iw)
        except:
            widget.layout.display= 'none'
            pass
        widget.children = children
            
            
    def update_view(self, handler, names = 'All', type = 'change', force = False):
        # DeltaShape
        if (handler and handler['name'] == "index") or force:
            c1 = self.compare_w.value[0]
            c2 = self.compare_w.value[1]
            
            ap = self.aptamer_w.value
            
            self.update_profile(c1, ap, self.sm_profile1)
            self.update_profile(c2, ap, self.sm_profile2)
            self.update_aggregated(c1, ap, self.sm_agg_profile1)
            self.update_aggregated(c2, ap, self.sm_agg_profile2)
            
            self.update_aggregated_full(c1, ap, self.sm_agg_full_profile1)
            self.update_aggregated_full(c2, ap, self.sm_agg_full_profile2)
            
            
            filepath = utils.delta_comp_pattern.format(path=self.project_path, condition1=c1, condition2=c2,conditions_separator="__", comp_prefix="comp_")

            try:
                with open(filepath, "rb") as file:
                    image = file.read()
                    self.deltashape.value = image
            except:
                self.deltashape.value = b''
                self.deltashape.layout.display= 'none'
    
            #self.update_structure(c1, ap, self.structure1)
            #self.update_structure(c2, ap, self.structure2)
            self.update_structure_box(c1, ap, self.structurebox1)
            self.update_structure_box(c2, ap, self.structurebox2)
         
        
    def __init__(self, data_path, conditions_prefix):
        self.project_path : str = data_path
        self.conditions_prefix: str = conditions_prefix
        self.conditions = utils.conditions_from_path(data_path, condition_prefix=conditions_prefix)
        self.rna_molecules = utils.get_sequences(data_path, self.conditions)
        self.comparisons = utils.comparisons_from_path(data_path, condition_prefix=conditions_prefix)
        
        if len(self.comparisons) == 0:
            self.comparisons = [ (cond1, cond2) for cond2 in self.conditions for cond1 in self.conditions if cond1 != cond2]
            
        self.init_view()
        self.init_events()
        self.update_view(None, force=True)

    def init_view(self):
        
        self.base_path = widgets.Text(
            value=self.project_path,
            description='Base Path',
            disabled=False)


        self.aptamer_w = widgets.Dropdown(
            options=self.rna_molecules
,
            description='RNA Molecules',)
        #self.condition1_w = widgets.Dropdown(
        #options={v: k for k, v in config.conditions.items()},
        #description='Condition 1',
        #)
    #
        #self.condition2_w = widgets.Dropdown(
        #    options={v: k for k, v in config.conditions.items()},
        #    description='Condition 2',
        #    value="aptamer-sam_1"
        #)
        
            
        self.compare_w = widgets.Dropdown(
               options={conds: conds for conds in self.comparisons},
            description='Conditions',
            #value=('SAMAP-nosam_Mg', 'SAMAP-sam_1_Mg')
        )
        
        self.sm_profile1 = widgets.Image(
        format='svg+xml',
        width=500,
        height=500,
        )
        
        self.sm_profile2 = widgets.Image(
        format='svg+xml',
        width=500,
        height=500,
        )
        self.sm_agg_full_profile1 = widgets.Image(
        format='svg+xml',
        width=1300,
        height=1000,
        )
        
        self.sm_agg_full_profile2 = widgets.Image(
        format='svg+xml',
        width=1300,
        height=1000,
        )
        self.sm_agg_profile1 = widgets.Image(
        format='svg+xml',
        width=1300,
        height=1000,
        )
        
        self.sm_agg_profile2 = widgets.Image(
        format='svg+xml',
        width=1300,
        height=1000,
        )
        
        
        self.deltashape = widgets.Image(
        format='svg+xml',
        width=1300,
        height=800,
        )
        
        self.structurebox1 = widgets.VBox([], width=500, height="auto")
        self.structurebox2 = widgets.VBox([], width=500, height="auto")
        
        self.structure1 = widgets.Image(
        format='svg+xml',
        width=800,
        height=600,
            
        )
        
        self.structure2 = widgets.Image(
        format='svg+xml',
        width=800,
        height=600,
        )
    
        self.prev_w= widgets.Button(
            description='Previous'
        )
        self.next_w = widgets.Button(
            description='Next'
        )
        
        self.profile_visible = widgets.Checkbox(
            value=True,
            description='Show profile',
            disabled=False,
            indent=False
        )
        self.deltashape_visible = widgets.Checkbox(
            value=True,
            description='Show deltashape',
            disabled=False,
            indent=False
        )
        self.structure_visible = widgets.Checkbox(
            value=True,
            description='Show structure',
            disabled=False,
            indent=False
        )
        
        
        #.layout.visibility = 'hidden'
        self.view = widgets.VBox(
        [
            widgets.HBox([ self.compare_w,self.aptamer_w,self.prev_w,self.next_w,self.profile_visible, self.deltashape_visible, self.structure_visible]),
            widgets.HBox([self.sm_profile1,self.sm_profile2]),
            widgets.VBox([self.sm_agg_profile1,self.sm_agg_profile2]),
            widgets.VBox([self.sm_agg_full_profile1,self.sm_agg_full_profile2]),
            self.deltashape,
            #widgets.HBox([self.structure1, self.structure2])
            widgets.HBox([self.structurebox1, self.structurebox2])
        ]
        )

    def toggle_visibility(self, cb, dst, h, n = 'All', t = 'change'):
        if cb.value:
            dst.layout.display = 'block'
        else:
            dst.layout.display= 'none'
        
    def next_aps(self, h, n = 'All', t = 'change'):
        cur = self.aptamer_w.index
        self.aptamer_w.value = self.aptamer_w.options[0 if cur >= len(self.aptamer_w.options) - 1 else cur + 1]

    def prev_aps(self, h, n = 'All', t = 'change'):
        cur = self.aptamer_w.index
        self.aptamer_w.value = self.aptamer_w.options[len(self.aptamer_w.options) - 1 if cur == 0 else cur - 1]
    
    
    def init_events(self):
        self.aptamer_w.observe(functools.partial(self.update_view))
        #self.condition1_w.observe(functools.partial(self.update_view))
        #self.condition2_w.observe(functools.partial(self.update_view))
        self.compare_w.observe(functools.partial(self.update_view))
        
        self.profile_visible.observe(functools.partial(self.toggle_visibility, self.profile_visible, self.sm_profile1))
        self.profile_visible.observe(functools.partial(self.toggle_visibility, self.profile_visible, self.sm_profile2))
        self.deltashape_visible.observe(functools.partial(self.toggle_visibility, self.deltashape_visible, self.deltashape))
        #self.structure_visible.observe(functools.partial(self.toggle_visibility, self.structure_visible, self.structure1))
        #self.structure_visible.observe(functools.partial(self.toggle_visibility, self.structure_visible, self.structure2))
        self.structure_visible.observe(functools.partial(self.toggle_visibility, self.structure_visible, self.structurebox1))
        self.structure_visible.observe(functools.partial(self.toggle_visibility, self.structure_visible, self.structurebox2))
        self.next_w.on_click(functools.partial(self.next_aps))
        self.prev_w.on_click(functools.partial(self.prev_aps))
        
    

In [52]:

Viewer(data_path, conditions_prefix).view

/mnt/nesrine/data/decrypted/shapemapper_results/SAMAP-norm-conds/SAMAP_1M7_noSAM_noMg_T37C_rep0/SAMAP_1M7_noSAM_noMg_T37C_rep0_APSAMN214_profiles.pdf
/mnt/nesrine/data/decrypted/shapemapper_results/SAMAP-norm-conds/SAMAP_1M7_noSAM_5Mg_T37C_rep0/SAMAP_1M7_noSAM_5Mg_T37C_rep0_APSAMN214_profiles.pdf


In [ ]:
/mnt/nesrine/data/decrypted/shapemapper_results/SAMAP-norm-conds/SAMAP_1M7_noSAM_5Mg_T37C_rep0/